In [93]:
import pandas as pd
import requests
import os
import json
import math

In [123]:
df = pd.read_csv("./output/developerAddress.csv")
first_half = df[:4000]
other_half = df[4000:]
other_half

,DEVELOPER_ADDRESS
4000,"84 Merton Street, Toronto, ON M4S 1A1"
4001,"Zeballos 313, Rosario"
4002,"601 Poydras St. Ste 1500\nNew Orleans, LA 70130"
4003,"Washington D.C., USA"
4004,"4655 Great America Parkway, Suite 300\nSanta C..."
...,...
8586,Viker Games Limited\n200 Union Street\nLondon\...
8587,"New York City, NY"
8588,"Украина, г.Сумы"
8589,Ankara/Turkey


In [125]:
from decouple import config
API_KEY = config('API_KEY')

In [136]:
requests_fired = 0;
for i in range(5000, len(other_half) + 4000):
    developer_addr = other_half.loc[i, "DEVELOPER_ADDRESS"]
    if (pd.isnull(developer_addr)):
        print("NaN")
        continue
    formmated = developer_addr.replace("#", " ")
    url = f"https://maps.googleapis.com/maps/api/place/autocomplete/json?input={formmated}&key={API_KEY}"
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    data = json.loads(response.text)
    if not 'predictions' in data.keys():
        other_half.at[i, "DEVELOPER_ADDRESS_PRED"] = "NaN"
        continue
    if (data["predictions"] == []):
        other_half.at[i, "DEVELOPER_ADDRESS_PRED"] = "undefined"
        continue
    prediction = data["predictions"][0]["description"]
    requests_fired +=1
    other_half.at[i, "DEVELOPER_ADDRESS"] = formmated
    other_half.at[i, "DEVELOPER_ADDRESS_PRED"] = prediction
other_half.head(20)

,DEVELOPER_ADDRESS,DEVELOPER_ADDRESS_PRED
4000,"84 Merton Street, Toronto, ON M4S 1A1","84 Merton Street, Toronto, ON M4S 1A1, Canada"
4001,"Zeballos 313, Rosario","Zeballos 313, Rosario, Santa Fe Province, Arge..."
4002,"601 Poydras St. Ste 1500\nNew Orleans, LA 70130","601 Poydras St, New Orleans, LA 70130, USA"
4003,"Washington D.C., USA","Washington D.C., DC, USA"
4004,"4655 Great America Parkway, Suite 300\nSanta C...","4655 Great America Parkway, Santa Clara, CA 95..."
4005,"Bangkok, Thailand","Bangkok, Thailand"
4006,"150-8554\nEbisu Subaru Bldg. 1-20-8, Ebisu, Sh...","Tully's Coffee Ebisu SUBARU Bldg. Shop, 1 Chom..."
4007,"REEF Technology\n601 Brickell Key Drive, Suite...",undefined
4008,"Via Fondacci, 269\n55054 Massarosa (LU)\nItaly","Via dei Fondacci, 269, 55054 Montramito, Massa..."
4009,Kingdom of Saudi Arabia,Kingdom of Saudi Arabia


In [139]:
file_path = "./other_half.csv"
other_half.to_csv(path_or_buf=file_path, index=False);

3012

In [140]:
first_half_df = pd.read_csv("./first_half.csv")
second_half_df = pd.read_csv("./other_half.csv")
combined_df = pd.concat([first_half_df, second_half_df])
combined_df.reset_index(drop=True, inplace=True)
combined_df

,DEVELOPER_ADDRESS,DEVELOPER_ADDRESS_PRED
0,23 Stoneleigh Avenue\nBN1 8NP,"23 Stoneleigh Avenue, Brighton BN1 8NP, UK"
1,"2500 Santa Clara Ave, Alameda, CA 94501","2500 Santa Clara Ave, Alameda, CA 94501, USA"
2,"Reading, Berkshire, RG6 3BT, United Kingdom","Sellafield Way, Lower Earley, Earley, Reading ..."
3,"2549-B Eastbluff Drive 201\nNewport Beach, CA...","2549 Eastbluff Drive, Newport Beach, CA 92660,..."
4,"POBOX 504905 DUBAI, UAE",Dubai - UAE
...,...,...
8586,Viker Games Limited\n200 Union Street\nLondon\...,"viker ltd, 200 Union Street, London SE1 0LX, UK"
8587,"New York City, NY","New York City, NY, USA"
8588,"Украина, г.Сумы","Сумы, Sumy Oblast, Ukraine"
8589,Ankara/Turkey,"Ankara, Turkey"


In [141]:
file_path = "./dev_addr.csv"
combined_df.to_csv(path_or_buf=file_path, index=False);

In [142]:
combined_df.head(10)

,DEVELOPER_ADDRESS,DEVELOPER_ADDRESS_PRED
0,23 Stoneleigh Avenue\nBN1 8NP,"23 Stoneleigh Avenue, Brighton BN1 8NP, UK"
1,"2500 Santa Clara Ave, Alameda, CA 94501","2500 Santa Clara Ave, Alameda, CA 94501, USA"
2,"Reading, Berkshire, RG6 3BT, United Kingdom","Sellafield Way, Lower Earley, Earley, Reading ..."
3,"2549-B Eastbluff Drive 201\nNewport Beach, CA...","2549 Eastbluff Drive, Newport Beach, CA 92660,..."
4,"POBOX 504905 DUBAI, UAE",Dubai - UAE
5,Güvenevler Mah.Keklik Cad.\nTürkiye,"Güvenevler, Cad Campus, Yenişehir/Mersin, Türkiye"
6,"12463 Rancho Bernardo Rd. 206, San Diego, CA ...","12463 Rancho Bernardo Road, San Diego, CA 9212..."
7,NaN,NaN
8,"4840 Hytech Drive\nCarson City, NV 89607","4840 Hytech Drive, Carson City, NV, USA"
9,"340/61 Quang Trung, Go Vap, Ho Chi Minh, Vietnam","340/61 Quang Trung, Gò Vấp, Ho Chi Minh City, ..."
